<a href="https://colab.research.google.com/github/teethavattcp/teethavat_data_sci_port/blob/main/Projects/01_PM_2.5_Forecasting_with_LSTM_RNN/PM2.5_Forecasting_BKK_01_extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandasql

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Clean data

## Explore data

In [ ]:
df_train_pm25 = pd.read_csv('/content/drive/MyDrive/DATA_SCI_SKILL/DATA SCI ENG/Final_Proj_1/datasci_dataset_2022/BKK/train/bkk_train.csv',sep=',')
df_test_pm25 = pd.read_csv('/content/drive/MyDrive/DATA_SCI_SKILL/DATA SCI ENG/Final_Proj_1/datasci_dataset_2022/BKK/test/bkk_test.csv',sep=',')

In [ ]:
df_train_pm25.head()

,Unnamed: 0,PM2.5
0,2017-07-01 00:00:00,14.0
1,2017-07-01 01:00:00,10.0
2,2017-07-01 02:00:00,17.0
3,2017-07-01 03:00:00,20.0
4,2017-07-01 04:00:00,15.0


In [ ]:
print('#train data =',df_train_pm25.shape[0])
print('#test data =',df_test_pm25.shape[0])

#train data = 26304
#test data = 8784


In [ ]:
df_train_pm25.isnull().sum()

Unnamed: 0      0
PM2.5         176
dtype: int64

In [ ]:
df_test_pm25.isnull().sum()

Unnamed: 0      0
PM2.5         970
dtype: int64

## Set time index

In [ ]:
def set_time_index(df):
  df.columns = ['Time','PM2.5(µg/m3)']
  df['Time'] = pd.to_datetime(df['Time'])
  df = df.set_index('Time')
  return df

In [ ]:
df_train_pm25 = set_time_index(df_train_pm25)
df_train_pm25

,PM2.5(µg/m3)
Time,
2017-07-01 00:00:00,14.0
2017-07-01 01:00:00,10.0
2017-07-01 02:00:00,17.0
2017-07-01 03:00:00,20.0
2017-07-01 04:00:00,15.0
...,...
2020-06-30 19:00:00,15.0
2020-06-30 20:00:00,14.0
2020-06-30 21:00:00,17.0


In [ ]:
df_test_pm25 = set_time_index(df_test_pm25)
df_test_pm25

,PM2.5(µg/m3)
Time,
2020-07-01 00:00:00,9.0
2020-07-01 01:00:00,8.0
2020-07-01 02:00:00,12.0
2020-07-01 03:00:00,9.0
2020-07-01 04:00:00,8.0
...,...
2021-07-01 19:00:00,13.0
2021-07-01 20:00:00,14.0
2021-07-01 21:00:00,14.0


## Handling missing values

In [ ]:
df_train_pm25['PM2.5(µg/m3)'].interpolate(method='time', limit_direction = 'both',inplace = True)
df_train_pm25.head()

,PM2.5(µg/m3)
Time,
2017-07-01 00:00:00,14.0
2017-07-01 01:00:00,10.0
2017-07-01 02:00:00,17.0
2017-07-01 03:00:00,20.0
2017-07-01 04:00:00,15.0


In [ ]:
df_test_pm25['PM2.5(µg/m3)'].interpolate(method='time', limit_direction = 'both',inplace = True)
df_test_pm25.head()

,PM2.5(µg/m3)
Time,
2020-07-01 00:00:00,9.0
2020-07-01 01:00:00,8.0
2020-07-01 02:00:00,12.0
2020-07-01 03:00:00,9.0
2020-07-01 04:00:00,8.0


In [ ]:
df_train_pm25.isnull().sum()

PM2.5(µg/m3)    0
dtype: int64

In [ ]:
df_test_pm25.isnull().sum()

PM2.5(µg/m3)    0
dtype: int64

## Filter time

From the limitation of extracting exogeneous data only being record for every 3 hours, time stamp need to be filtered.

In [ ]:
time_filter = {1:'18', 4:'21', 7:'00',10:'03',
               13:'06', 16:'09', 19:'12', 22:'15'}
#{key = local time, value = hh index in url [UTC time]}

In [ ]:
list(time_filter.keys())

[1, 4, 7, 10, 13, 16, 19, 22]

In [ ]:
df = df_train_pm25.copy()
df.head()

,PM2.5(µg/m3)
Time,
2017-07-01 00:00:00,14.0
2017-07-01 01:00:00,10.0
2017-07-01 02:00:00,17.0
2017-07-01 03:00:00,20.0
2017-07-01 04:00:00,15.0


In [ ]:
def filter_time(df):
  df["hour"] = df.index.hour
  df = pd.concat([
    df.loc[df["hour"].isin(x)]
    for x in [list(time_filter.keys())]])
  df.drop(columns= ['hour'],inplace=True)
  return df

In [ ]:
df_train_pm25 = filter_time(df_train_pm25)
df_train_pm25

,PM2.5(µg/m3)
Time,
2017-07-01 01:00:00,10.0
2017-07-01 04:00:00,15.0
2017-07-01 07:00:00,23.0
2017-07-01 10:00:00,22.0
2017-07-01 13:00:00,14.0
...,...
2020-06-30 10:00:00,14.0
2020-06-30 13:00:00,12.0
2020-06-30 16:00:00,9.0


In [ ]:
df_test_pm25 = filter_time(df_test_pm25)
df_test_pm25

,PM2.5(µg/m3)
Time,
2020-07-01 01:00:00,8.0
2020-07-01 04:00:00,8.0
2020-07-01 07:00:00,13.0
2020-07-01 10:00:00,10.0
2020-07-01 13:00:00,15.0
...,...
2021-07-01 10:00:00,14.0
2021-07-01 13:00:00,14.0
2021-07-01 16:00:00,14.0


## Change timestamp from local to UTC


Website url use UTC time as index

In [ ]:
def local_id_to_utc_id(df):
  df_ = df.copy()
  df_['Time_utc'] = df_.index.tz_localize("Asia/Bangkok").tz_convert("UTC")
  df_['Time_utc'] = df_['Time_utc'].dt.tz_localize(None)
  df_ = df_.set_index('Time_utc')
  return df_

In [ ]:
df_train_pm25_utc = local_id_to_utc_id(df_train_pm25)
df_train_pm25_utc

,PM2.5(µg/m3)
Time_utc,
2017-06-30 18:00:00,10.0
2017-06-30 21:00:00,15.0
2017-07-01 00:00:00,23.0
2017-07-01 03:00:00,22.0
2017-07-01 06:00:00,14.0
...,...
2020-06-30 03:00:00,14.0
2020-06-30 06:00:00,12.0
2020-06-30 09:00:00,9.0


In [ ]:
df_test_pm25_utc = local_id_to_utc_id(df_test_pm25)
df_test_pm25_utc

,PM2.5(µg/m3)
Time_utc,
2020-06-30 18:00:00,8.0
2020-06-30 21:00:00,8.0
2020-07-01 00:00:00,13.0
2020-07-01 03:00:00,10.0
2020-07-01 06:00:00,15.0
...,...
2021-07-01 03:00:00,14.0
2021-07-01 06:00:00,14.0
2021-07-01 09:00:00,14.0


# Extract missing exogeneous data

## Get coordinations of BKK from position.txt

In [ ]:
lat_long = ()
infile = open('/content/drive/MyDrive/DATA_SCI_SKILL/DATA SCI ENG/Final_Proj_1/datasci_dataset_2022/BKK/position.txt','r')
for line in infile:
  lat_long += (float(line.strip()[int(line.find('='))+1:]),)
infile.close()
print('BKK coordinate =', lat_long)

BKK coordinate = (13.729984, 100.536443)


In [ ]:
long = str(round(lat_long[1],3))
lat = str(round(lat_long[0],3))+'0'
print(lat,long)

13.730 100.536


## Scrap data from https://earth.nullschool.net/

### Import required libraries for data extraction

In [ ]:
!pip install -U selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install webdriver-manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.12
    Uninstalling urllib3-1.26.12:
      Successfully uninstalled urllib3-1.26.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.5.0 requires urllib3[socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.


In [ ]:
!pip install kora -q

In [ ]:
from kora.selenium import wd as driver

In [ ]:
from selenium.webdriver.common.by import By
ID = "id"
NAME = "name"
XPATH = "xpath"
LINK_TEXT = "link text"
PARTIAL_LINK_TEXT = "partial link text"
TAG_NAME = "tag name"
CLASS_NAME = "class name"
CSS_SELECTOR = "css selector"

In [ ]:
import time

### Get exogeneous data

Previous studies (H. Zhang et al., 2015; Pearce et al., 2011; Yadav et al., 2014) revealed that such meteorological factors as relative humidity, temperature, wind speed, and wind direction might be related to PM2.5 concentrations.

Mode: Wind 850

```
https://earth.nullschool.net/#2022/09/26/0600Z/wind/isobaric/850hPa/orthographic=-257.89,7.85,1631/loc=100.536,13.730
```


Mode: Temp @surface
```
https://earth.nullschool.net/#2022/09/26/0600Z/wind/surface/level/overlay=temp/orthographic=-257.89,7.85,1631/loc=100.536,13.730
```

Mode: RH
```
https://earth.nullschool.net/#2022/09/26/0600Z/wind/surface/level/overlay=relative_humidity/orthographic=-257.89,7.85,1631/loc=100.536,13.730
```

Website format:

```
https://earth.nullschool.net/#yyyy/mm/dd/hh.../loc=long,lat
```

Variable:
```
yyyy = year in C.E.
mm = month
dd = day
long = longitude
lat = latitude

```

In [ ]:
print('lat =', lat, ', long =', long)

lat = 13.730 , long = 100.536


for loop scrap data by (coordinate)--->date--->mode: 

edit website link by date + mode, 

In [ ]:
df = df_train_pm25_utc.copy()

In [ ]:

yyyy_mm_dd = list(df.index.strftime("%Y/%m/%d").drop_duplicates())
print('yyyy_mm_dd = [')
for i in range(3):
  print(yyyy_mm_dd[i],',')
print('...]')
print('#element =', len(yyyy_mm_dd))

yyyy_mm_dd = [
2017/06/30 ,
2017/07/01 ,
2017/07/02 ,
...]
#element = 1097


#### Define scraping function

Given that there were data of temperature, wind direction and wind speed for train set but none for test set.

!!! Don't forget to delay time in for-loop of filling extracted data thanks to problem of website unloading just in time

In [ ]:
#earth_bt = driver.find_element(By.CSS_SELECTOR, 'div.earth-bar > div > h1 > button')
def if_click_earth_bt():
  #if the menu was hidden then click earth button o/w do nothing
  global earth_bt
  earth_bt = driver.find_element(By.CSS_SELECTOR, 'div.earth-bar > div > h1 > button')
  if driver.find_element(By.ID,"menu").get_attribute('hidden') == 'true':
        earth_bt.click()

In [ ]:
#navi_bt = driver.find_element(By.XPATH, '//*[@id="menu"]/div/div[5]/div[3]/div[2]/button[4]')
def if_click_navi_bt():
    #if the spot was hidden then click navi button o/w do nothing
    global navi_bt
    navi_bt = driver.find_element(By.XPATH, '//*[@id="menu"]/div/div[5]/div[3]/div[2]/button[4]')
    if driver.find_element(By.ID,"spotlight-panel").get_attribute('hidden') == 'true':
        navi_bt.click()

In [ ]:
sorted(list(time_filter.values()))

['00', '03', '06', '09', '12', '15', '18', '21']

In [ ]:
def scrape_exo(df):
  yyyy_mm_dd = list(df.index.strftime("%Y/%m/%d").drop_duplicates())
  list_wind_dir = []
  list_wind_speed = []
  list_temp = []
  list_rh = []
  time_delay = 0.55 #sec
  list_time_stamp_utc = []
  for i in range(len(yyyy_mm_dd)):
    for hh in sorted(list(time_filter.values())):
      date_utc = yyyy_mm_dd[i] + ' ' + hh + ":00:00"
      list_time_stamp_utc.append(date_utc)

      #wind at height 850 hPa
      url_wind = 'https://earth.nullschool.net/#'+yyyy_mm_dd[i]+'/' +hh+'00Z/wind/isobaric/850hPa/orthographic=-257.89,7.85,1631/loc='+long+','+lat
      driver.get(url=url_wind)
      time.sleep(time_delay)
      if_click_earth_bt()
      time.sleep(time_delay)
      if_click_navi_bt()
      time.sleep(time_delay)
      extract_wind = driver.find_element(By.XPATH,'//*[@id="spotlight-panel"]/div[2]/div').text
      wind_dir = extract_wind[:extract_wind.find('°')].strip()
      wind_speed = extract_wind[extract_wind.find('@')+1:].strip()
      list_wind_dir.append(wind_dir)
      list_wind_speed.append(wind_speed)


      #temp at surface
      url_temp = 'https://earth.nullschool.net/#'+yyyy_mm_dd[i]+'/' +hh+'00Z/wind/surface/level/overlay=temp/orthographic=-257.89,7.85,1631/loc='+long+','+lat
      driver.get(url=url_temp)
      time.sleep(time_delay)
      if_click_earth_bt()
      time.sleep(time_delay)
      if_click_navi_bt()
      time.sleep(time_delay)
      temp = driver.find_element(By.XPATH,'//*[@id="spotlight-panel"]/div[3]/div').text.strip()
      list_temp.append(temp)

      #relative humidity at surface
      url_rh = 'https://earth.nullschool.net/#'+yyyy_mm_dd[i]+'/' +hh+'00Z/wind/surface/level/overlay=relative_humidity/orthographic=-257.89,7.85,1631/loc='+long+','+lat
      driver.get(url=url_rh)
      time.sleep(time_delay)
      if_click_earth_bt()
      time.sleep(time_delay)
      if_click_navi_bt()
      time.sleep(time_delay)
      rh = driver.find_element(By.XPATH,'//*[@id="spotlight-panel"]/div[3]/div').text.strip()
      list_rh.append(rh)
  
  data_exo ={
    'Time (UTC)': list_time_stamp_utc,
    'Wind Speed (km/h)': list_wind_speed,
    'Wind Direction (°)': list_wind_dir,
    'Surface Temperature (°C)': list_temp,
    'Surface Relative Humidity (%)': list_rh
    }

  df_exo = pd.DataFrame(data_exo)
  return df_exo

In [ ]:
def scrape_exo_train(df):
  yyyy_mm_dd = list(df.index.strftime("%Y/%m/%d").drop_duplicates())
  list_rh = []
  time_delay = 0.5 #sec
  list_time_stamp_utc = []
  for i in range(len(yyyy_mm_dd)):
    for hh in sorted(list(time_filter.values())):
      date_utc = yyyy_mm_dd[i] + ' ' + hh + ":00:00"
      list_time_stamp_utc.append(date_utc)

      #relative humidity at surface
      url_rh = 'https://earth.nullschool.net/#'+yyyy_mm_dd[i]+'/' +hh+'00Z/wind/surface/level/overlay=relative_humidity/orthographic=-257.89,7.85,1631/loc='+long+','+lat
      driver.get(url=url_rh)
      time.sleep(time_delay)
      if_click_earth_bt()
      time.sleep(time_delay)
      if_click_navi_bt()
      time.sleep(time_delay)
      rh = driver.find_element(By.XPATH,'//*[@id="spotlight-panel"]/div[3]/div').text.strip()
      list_rh.append(rh)
  
  data_exo ={
      'Time (UTC)': list_time_stamp_utc,
      'Surface Relative Humidity (%)': list_rh
    }

  df_exo = pd.DataFrame(data_exo)
  return df_exo

#### Extract data in train and test set

In [ ]:
df_exo_test = scrape_exo(df_test_pm25_utc) #.iloc[:50]

In [ ]:
df_exo_test

,Time (UTC),Wind Speed (km/h),Wind Direction (°),Surface Temperature (°C),Surface Relative Humidity (%)
0,2020/06/30 00:00:00,22,255,28.5,71
1,2020/06/30 03:00:00,24,250,30.8,61
2,2020/06/30 06:00:00,23,250,32.0,60
3,2020/06/30 09:00:00,37,255,32.1,61
4,2020/06/30 12:00:00,36,265,30.3,69
...,...,...,...,...,...
2931,2021/07/01 09:00:00,19,240,32.5,52
2932,2021/07/01 12:00:00,18,250,29.1,70
2933,2021/07/01 15:00:00,16,245,29.1,71
2934,2021/07/01 18:00:00,17,260,28.6,73


In [ ]:
df_exo_test.isnull().sum()

Time (UTC)                       0
Wind Speed (km/h)                0
Wind Direction (°)               0
Surface Temperature (°C)         0
Surface Relative Humidity (%)    0
dtype: int64

In [ ]:
df_exo_train = scrape_exo_train(df_train_pm25_utc) #.iloc[:50]

In [ ]:
df_exo_train

,Time (UTC),Surface Relative Humidity (%)
0,2017/06/30 00:00:00,74
1,2017/06/30 03:00:00,62
2,2017/06/30 06:00:00,55
3,2017/06/30 09:00:00,55
4,2017/06/30 12:00:00,72
...,...,...
8771,2020/06/30 09:00:00,61
8772,2020/06/30 12:00:00,69
8773,2020/06/30 15:00:00,69
8774,2020/06/30 18:00:00,68


In [ ]:
df_exo_train.isnull().sum()

Time (UTC)                       0
Surface Relative Humidity (%)    0
dtype: int64

# Export .csv

In [ ]:
from pathlib import Path 

In [ ]:
def export_scrap_to_csv(scrap_file_name,df):
  filepath = Path('/content/drive/MyDrive/DATA_SCI_SKILL/DATA SCI ENG/Final_Proj_1/'+str(scrap_file_name)+'_rescrap_BKK.csv')
  filepath.parent.mkdir(parents=True, exist_ok=True)  
  df.to_csv(filepath) 

In [ ]:
def export_pm25_to_csv(pm25_file_name,df):
  filepath = Path('/content/drive/MyDrive/DATA_SCI_SKILL/DATA SCI ENG/Final_Proj_1/'+str(pm25_file_name)+'_pm25_BKK.csv')
  filepath.parent.mkdir(parents=True, exist_ok=True)  
  df.to_csv(filepath) 

In [ ]:
export_scrap_to_csv('test_all',df_exo_test)

In [ ]:
export_scrap_to_csv('train_all',df_exo_train)

In [ ]:
export_pm25_to_csv('test',df_test_pm25)

In [ ]:
export_pm25_to_csv('train',df_train_pm25)